# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg_agent import Agent
from buffer import ReplayBuffer
from collections import deque,namedtuple
import torch
import torch.nn.functional as F
from maddpg_agent import MADDPG

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode = False)[brain_name]
print(env_info.vector_observations)

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.65278625 -1.5
  -0.          0.          6.83172083  6.         -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.4669857  -1.5
   0.          0.         -6.83172083  6.          0.          0.        ]]
Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 2):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        print(actions)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        print(next_states)
        rewards = env_info.rewards                         # get reward (for each agent)
        print(rewards)
        dones = env_info.local_done                        # see if episode finished
        print(dones)
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))                                        
    

[[0.00334968 1.        ]
 [0.10860023 0.56719579]]
[[ 0.          0.          0.          0.          0.          0.
   0.          0.         -7.38993645 -1.5        -0.          0.
   6.83172083  5.99607611 -0.          0.         -7.3798871  -0.98316395
   0.10049034  6.21520042  6.83172083  5.91759634  0.10049034  6.21520042]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.         -6.70024681 -1.5         0.          0.
  -6.83172083  5.99607611  0.          0.         -6.37444544 -0.98316395
   3.25800681  6.21520042 -6.83172083  5.91759634  3.25800681  6.21520042]]
[0.0, 0.0]
[False, False]
[[ 0.14187236  0.52576424]
 [-0.99707873 -0.61814165]]
[[ -7.38993645  -1.5         -0.           0.           6.83172083
    5.99607611  -0.           0.          -7.3798871   -0.98316395
    0.10049034   6.21520042   6.83172083   5.91759634   0.10049034
    6.21520042  -6.95426941  -0.42050385   4.25617075   5.23420095
    6.83172083   5.74101639   4.25617

[[ -4.94082737  -0.18940151   6.98115158  -4.57579613   6.83172083
   -1.00070286   6.98115158  -4.57579613  -7.94082689  -0.70584118
  -30.          -5.5567956    6.83172083  -1.92032266 -30.
   -5.5567956  -10.94082928  -1.32038057 -30.          -6.53779507
    6.83172083  -2.83994246 -30.          -6.53779507]
 [ -2.47840619  -0.18940151  16.27853775  -4.57579613  -6.83172083
   -1.00070286  16.27853775  -4.57579613  -0.79284996  -0.70584118
   16.8555603   -5.5567956   -6.83172083  -1.92032266  16.8555603
   -5.5567956   -0.76293987  -1.32038057  11.85300636  -6.53779507
   -6.83172083  -2.83994246  11.85300636  -6.53779507]]
[0.0, 0.0]
[False, False]
[[-1.          0.41953963]
 [-0.92204188  0.06262827]]
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.43639946 -1.5
  -0.          0.          6.69487906  5.96076012 -0.          0.        ]
 [ 0.  

When finished, you can close the environment.

In [6]:
def vector_avg(states):
    state_1 = np.array(states[0])
    state_2 = np.array(states[1])
    [state_1_1, state_1_2, state_1_3] = np.split(state_1, 3)
    [state_2_1, state_2_2, state_2_3] = np.split(state_2, 3)
    state_1 = (state_1_1 + state_1_2 + state_1_3)/3
    state_2 = (state_2_1 + state_2_2 + state_2_3)/3
    return [state_1, state_2]
    


state = vector_avg(states)
print(state[0])

print(state[1])


[-2.47879982 -0.5         0.          0.          2.23162635  1.98692004
  0.          0.        ]
[-2.57673184 -0.5         0.          0.         -2.23162635  1.98692004
  0.          0.        ]


In [9]:
maddpg = MADDPG(24,2)

def main():
    print_every = 10
    scores_deque = deque(maxlen = 100)
    scores = []
    num_episodes = 5000
    episode_length = 80
    batchsize = 256
    noise = 2
    noise_reduction = 0.9999
    t_max = 300
    for episode in range(0,num_episodes):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        [agent.reset() for agent in maddpg.maddpg]
        score = 0
        for t in range(t_max):
            actions = maddpg.act((states), noise = noise)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            score += max(env_info.rewards)
            print(rewards)
            maddpg.step(states, actions, rewards, next_states, dones)
            states = next_states
            if np.any(dones):                                  # exit loop if episode finished
                break
        scores_deque.append(score)
        scores.append(score)
        print("\r Episode {}\tAverage Score: {:.2f}".format(episode, np.mean(scores_deque)), end = "")
        torch.save(maddpg.maddpg[0].actor_local.state_dict(), "checkpoint_actor_agent1.pth")
        torch.save(maddpg.maddpg[0].critic_local.state_dict(), "checkpoint_critic_agent1.pth")
        torch.save(maddpg.maddpg[1].actor_local.state_dict(), "checkpoint_actor_agent2.pth")
        torch.save(maddpg.maddpg[1].critic_local.state_dict(), "checkpoint_critic_agent2.pth")
scores = main()

[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[-0.009999999776482582, 0.0]
 Episode 0	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, -0.009999999776482582]
 Episode 1	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, -0.009999999776482582]
 Episode 2	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[-0.009999999776482582, 0.0]
 Episode 3	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[-0.009999999776482582, 0.0]
 Episode 4	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[

[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, -0.009999999776482582]
 Episode 40	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[-0.009999999776482582, 0.0]
 Episode 41	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[-0.009999999776482582, 0.0]
 Episode 42	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[-0.009999999776482582, 0.0]
 Episode 43	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[-0.009999999776482582, 0.0]
 Episode 44	Average Score: 0.00[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0

KeyboardInterrupt: 

In [ ]:
print(actions)
states = states.reshape(1,-1)

print(states.reshape(1, -1).shape)
states = states.reshape(-1, 2, 24)

critic_input = np.concatenate((states[0], actions[0]), axis = None)
print(critic_input)
print(scores_deque)

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```